In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from chroma.db.clickhouse import Clickhouse
from chroma.db.index.hnswlib import Hnswlib
from chroma.config import Settings

from chroma.utils.sampling import *

In [2]:
chroma_settings = Settings()
db = Clickhouse(chroma_settings)
model_space = "default_scope"

training_dataset_name = "waymo-phx-train-hc"
unlabeled_dataset_name = "waymo-sf-val-hc"

training_data = db.fetch(
    where={"model_space": model_space, "dataset": training_dataset_name}
)
unlabeled_data = db.fetch(
    where={"model_space": model_space, "dataset": unlabeled_dataset_name}
)

ann_index = Hnswlib(chroma_settings)

ann_index.run(
    model_space=model_space,
    uuids=training_data["uuid"].tolist(),
    embeddings=training_data["embedding"].tolist(),
    num_threads=8
)

time to fetch 1677402 embeddings:  7.980594158172607
time to fetch 205777 embeddings:  1.0896880626678467


# Score and Store

In [3]:
score_and_store(
    training_dataset_name=training_dataset_name,
    unlabeled_dataset_name=unlabeled_dataset_name,
    db_connection=db,
    ann_index=ann_index,
    model_space=model_space,
)

time to fetch 1677402 embeddings:  7.561792850494385
time to fetch 205777 embeddings:  0.8599541187286377


# Get Samples

In [4]:
sample_proportions = {
    "activation_uncertainty": 0.3,
    "boundary_uncertainty": 0.3,
    "representative_cluster_outlier": 0.2,
    "random": 0.2,
}

samples = get_sample(dataset_name=unlabeled_dataset_name, n_samples=1000, sample_proportions=sample_proportions, db_connection=db, model_space=model_space)